In [1]:
# 크롤링 및 추천시스템 위한 라이브러리 호출
import pandas as pd  
import numpy as np   

import requests
from bs4 import BeautifulSoup    
from selenium import webdriver   
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller 

import time    
import re
import math
from tqdm import tqdm_notebook

# 데이터분석 위한 라이브러리 호출

import nltk
from konlpy.tag import Okt

# 그래프
%matplotlib inline
import os
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz             
from sklearn.tree import export_graphviz

# matplotlib 그래프 한글폰트 깨질 때 대처(Mac & Window)
import matplotlib
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/HMFMMUEX.ttc").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
# 그래프에서 마이너스 기호가 표시되도록 하는 설정
matplotlib.rcParams['axes.unicode_minus'] = False

from collections import Counter

import warnings
warnings.filterwarnings('ignore')


# 추천시스템

## CBF 카테고리 기반 추천시스템

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
# 가중평점 반영 안한 추천시스템
def find_sim_book_ver1(gyobo_CBF, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_book = gyobo_CBF[gyobo_CBF['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_book.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임.
    # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)    
    # 2차원 데이터를 1차원으로 변환
    similar_indexes = similar_indexes.reshape(-1)
    
    return gyobo_CBF.iloc[similar_indexes]

In [35]:
# 가중평점 반영한 추천시스템
def find_sim_book_ver2(gyobo_CBF, sorted_ind, title_name, top_n=10):
    title_book = gyobo_CBF[gyobo_CBF['title'] == title_name]
    title_index = title_book.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출
    similar_indexes = sorted_ind[title_index, :(top_n*1)]
    similar_indexes = similar_indexes.reshape(-1)

    # 기준 서적 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return gyobo_CBF.iloc[similar_indexes].sort_values('weighted_score', ascending=False)[:top_n]

In [36]:
# 필요한 컬럼만 추출
gyobo_CBF = gyobo[['title', 'keyword', 'score', 'total']]

# 중복데이터 제거
gyobo_CBF = gyobo_CBF.drop_duplicates(subset="title", keep='first', inplace=False, ignore_index=False)
gyobo_CBF.reset_index(drop=True, inplace=True)
gyobo_CBF

,title,keyword,score,total
0,첫 번째 디자인 케이크,디저트 초코 아이싱 플라워 제과제빵 밀크티,9.2,15
1,연금술사,마음 모험 연금술 만물 성장소설 영혼,9.6,350
2,그때 그 마음(2022 제67회 현대문학상 수상소설집),순정 혜성 초파리 심사 폐허 허공,10.0,5
3,대한민국 금기 깨기,부총리 전어 기회 반란 디지털 경제 국민,9.4,37
4,하루 한 장 아이패드 드로잉,에세이 그림 인스타그램 그리기 도안 일러스트,9.9,17
...,...,...,...,...
3234,스물아홉 생일 1년 후 죽기로 결심했다,시한부 자전적에세이 라스베이거스 선고 치열 외톨이,9.2,368
3235,Bricks Easy Listening 100. 2,NaN,10.0,2
3236,백조와 박쥐,일본소설 살인사건 미스터리소설 작품 미스터리 일본 추리 소설,9.7,211
3237,스포츠심리학의 정석(개정판),불안 모범 답안 실습 과제 평가 문항 효능감 복습,10.0,2


In [37]:
# 결측치 확인 및 결측치 제거
gyobo_CBF.isnull().sum()
# 결측치 삭제
gyobo_CBF = gyobo_CBF.dropna(axis=0)

In [38]:
gyobo_CBF

,title,keyword,score,total
0,첫 번째 디자인 케이크,디저트 초코 아이싱 플라워 제과제빵 밀크티,9.2,15
1,연금술사,마음 모험 연금술 만물 성장소설 영혼,9.6,350
2,그때 그 마음(2022 제67회 현대문학상 수상소설집),순정 혜성 초파리 심사 폐허 허공,10.0,5
3,대한민국 금기 깨기,부총리 전어 기회 반란 디지털 경제 국민,9.4,37
4,하루 한 장 아이패드 드로잉,에세이 그림 인스타그램 그리기 도안 일러스트,9.9,17
...,...,...,...,...
3233,그러라 그래(양장본 HardCover),인생 에세이 삶 김나영 여성 시대 아이유,9.4,180
3234,스물아홉 생일 1년 후 죽기로 결심했다,시한부 자전적에세이 라스베이거스 선고 치열 외톨이,9.2,368
3236,백조와 박쥐,일본소설 살인사건 미스터리소설 작품 미스터리 일본 추리 소설,9.7,211
3237,스포츠심리학의 정석(개정판),불안 모범 답안 실습 과제 평가 문항 효능감 복습,10.0,2


In [39]:
# 카테고리(= 책 장르 컬럼 추가)
gyobo_CBF = gyobo_CBF.join(gyobo_best.set_index('title')['카테고리'], on='title')

In [40]:
# 카테고리와 장르 합친 새로운 컬럼 생성
gyobo_CBF["keyword2"] = gyobo_CBF["카테고리"] + " " + gyobo_CBF["keyword"]
# 중복데이터 제거
gyobo_CBF = gyobo_CBF.drop_duplicates(subset="title", keep='first', inplace=False, ignore_index=False)
gyobo_CBF.reset_index(drop=True, inplace=True)
gyobo_CBF

,title,keyword,score,total,카테고리,keyword2
0,첫 번째 디자인 케이크,디저트 초코 아이싱 플라워 제과제빵 밀크티,9.2,15,요리,요리 디저트 초코 아이싱 플라워 제과제빵 밀크티
1,연금술사,마음 모험 연금술 만물 성장소설 영혼,9.6,350,소설,소설 마음 모험 연금술 만물 성장소설 영혼
2,그때 그 마음(2022 제67회 현대문학상 수상소설집),순정 혜성 초파리 심사 폐허 허공,10.0,5,소설,소설 순정 혜성 초파리 심사 폐허 허공
3,대한민국 금기 깨기,부총리 전어 기회 반란 디지털 경제 국민,9.4,37,정치사회,정치사회 부총리 전어 기회 반란 디지털 경제 국민
4,하루 한 장 아이패드 드로잉,에세이 그림 인스타그램 그리기 도안 일러스트,9.9,17,예술대중문화,예술대중문화 에세이 그림 인스타그램 그리기 도안 일러스트
...,...,...,...,...,...,...
3091,그러라 그래(양장본 HardCover),인생 에세이 삶 김나영 여성 시대 아이유,9.4,180,시/에세이,시/에세이 인생 에세이 삶 김나영 여성 시대 아이유
3092,스물아홉 생일 1년 후 죽기로 결심했다,시한부 자전적에세이 라스베이거스 선고 치열 외톨이,9.2,368,시/에세이,시/에세이 시한부 자전적에세이 라스베이거스 선고 치열 외톨이
3093,백조와 박쥐,일본소설 살인사건 미스터리소설 작품 미스터리 일본 추리 소설,9.7,211,소설,소설 일본소설 살인사건 미스터리소설 작품 미스터리 일본 추리 소설
3094,스포츠심리학의 정석(개정판),불안 모범 답안 실습 과제 평가 문항 효능감 복습,10.0,2,취미실용스포츠,취미실용스포츠 불안 모범 답안 실습 과제 평가 문항 효능감 복습


In [41]:
# CountVectorizer로 학습시켰더니 3097개의 책에 대한 22885개의 키워드의 "키워드 매트릭스"가 생성되었다.

count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))  # min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 2 1단어, 2단어까지
genre_mat = count_vect.fit_transform(gyobo_CBF['keyword']) # 키워드 기반학습

In [42]:
gyobo_CBF = gyobo_CBF.dropna(axis=0)

In [43]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat2 = count_vect.fit_transform(gyobo_CBF['keyword2']) # 키워드2 기반 학습

In [44]:
# 키워드 기반 학습
print(genre_mat.shape)
print(genre_mat)

(3096, 22882)
  (0, 4727)	1
  (0, 18779)	1
  (0, 12101)	1
  (0, 20950)	1
  (0, 17168)	1
  (0, 6916)	1
  (0, 4734)	1
  (0, 18780)	1
  (0, 12103)	1
  (0, 20953)	1
  (0, 17169)	1
  (1, 5428)	1
  (1, 6123)	1
  (1, 13218)	1
  (1, 5558)	1
  (1, 9827)	1
  (1, 13591)	1
  (1, 5438)	1
  (1, 6137)	1
  (1, 13219)	1
  (1, 5559)	1
  (1, 9831)	1
  (2, 10633)	1
  (2, 22370)	1
  (2, 18790)	1
  :	:
  (3094, 6026)	1
  (3094, 3828)	1
  (3094, 6027)	1
  (3094, 8283)	1
  (3094, 11668)	1
  (3094, 1399)	1
  (3094, 22729)	1
  (3094, 8284)	1
  (3094, 3830)	1
  (3094, 11669)	1
  (3094, 1401)	1
  (3094, 20529)	1
  (3094, 6558)	1
  (3094, 22730)	1
  (3095, 16886)	1
  (3095, 1200)	1
  (3095, 1558)	1
  (3095, 10306)	1
  (3095, 20997)	1
  (3095, 13532)	1
  (3095, 16887)	1
  (3095, 1224)	1
  (3095, 13538)	1
  (3095, 10307)	1
  (3095, 1561)	1


In [45]:
# 키워드2(카테고리 + 키워드) 기반 학습
print(genre_mat2.shape)
print(genre_mat2)

(3095, 24435)
  (0, 14845)	1
  (0, 5019)	1
  (0, 20133)	1
  (0, 12598)	1
  (0, 22453)	1
  (0, 18267)	1
  (0, 7255)	1
  (0, 14856)	1
  (0, 5026)	1
  (0, 20134)	1
  (0, 12600)	1
  (0, 22456)	1
  (0, 18268)	1
  (1, 10497)	1
  (1, 5720)	1
  (1, 6462)	1
  (1, 13907)	1
  (1, 5850)	1
  (1, 10229)	1
  (1, 14280)	1
  (1, 10516)	1
  (1, 5730)	1
  (1, 6476)	1
  (1, 13908)	1
  (1, 5851)	1
  :	:
  (3093, 8685)	1
  (3093, 12165)	1
  (3093, 1617)	1
  (3093, 24282)	1
  (3093, 20370)	1
  (3093, 8686)	1
  (3093, 4122)	1
  (3093, 12166)	1
  (3093, 1619)	1
  (3093, 22032)	1
  (3093, 6897)	1
  (3093, 24283)	1
  (3094, 14743)	1
  (3094, 17893)	1
  (3094, 14787)	1
  (3094, 1344)	1
  (3094, 1849)	1
  (3094, 10734)	1
  (3094, 22500)	1
  (3094, 14221)	1
  (3094, 17894)	1
  (3094, 1368)	1
  (3094, 14227)	1
  (3094, 10735)	1
  (3094, 1852)	1


In [46]:
# 코사인 유사도에 의해 3096개 리스트에 대한 유사한 책 계산
from sklearn.metrics.pairwise import cosine_similarity
genre_sim = cosine_similarity(genre_mat, genre_mat)
genre_sim2 = cosine_similarity(genre_mat2, genre_mat2)

print(genre_sim.shape) # 키워드 기반
print(genre_sim[:10])

print(genre_sim2.shape) # 키워드2(카테고리+키워드) 기반
print(genre_sim2[:10])

(3096, 3096)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(3095, 3095)
[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.07692308 ... 0.12725695 0.         0.        ]
 [0.         0.07692308 1.         ... 0.12725695 0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.07161149]]


In [47]:
# 자료를 정렬하는 것이 아니라 순서만 알고 싶다면 argsort
# 유사도가 높은 책을 앞에서부터 순서대로 보여줌

# 0번째 책의 경우 유사도 순서 : 0번, 3494번, 813번, ..., 2401 순서
# 키워드 기반
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1] # ::-1 : 역순으로 정렬
print(genre_sim_sorted_ind[:5])

#키워드2 기반
genre_sim_sorted_ind2 = genre_sim2.argsort()[:, ::-1] # ::-1 : 역순으로 정렬
print(genre_sim_sorted_ind[:5])

[[   0  791 2615 ... 2055 2054 1547]
 [   1  653 2165 ... 2049 2048    0]
 [   2 2256 1088 ... 2058 2057    0]
 [   3 2875  263 ... 2054 2053    0]
 [   4 2220 1422 ... 2043 2042    0]]
[[   0  791 2615 ... 2055 2054 1547]
 [   1  653 2165 ... 2049 2048    0]
 [   2 2256 1088 ... 2058 2057    0]
 [   3 2875  263 ... 2054 2053    0]
 [   4 2220 1422 ... 2043 2042    0]]


In [48]:
print(genre_sim_sorted_ind)

print(genre_sim_sorted_ind2)

[[   0  791 2615 ... 2055 2054 1547]
 [   1  653 2165 ... 2049 2048    0]
 [   2 2256 1088 ... 2058 2057    0]
 ...
 [3093  897  949 ... 2029 2028    0]
 [3094  192 1059 ... 2042 2041    0]
 [3095  671 2080 ... 2039 2038    0]]
[[   0  791 1715 ... 2027 2026 1547]
 [   1 2164 1013 ... 2022 2021    0]
 [   2 2659 2885 ... 2033 2032    0]
 ...
 [3092  949  897 ... 2016 2015    0]
 [3093  192  792 ... 2021 2020    0]
 [3094 1057  671 ... 2020 2018    0]]


In [49]:
# 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
title_book = gyobo_CBF[gyobo_CBF['title'] == '스포츠심리학의 정석(개정판)']
title_book

,title,keyword,score,total,카테고리,keyword2
3094,스포츠심리학의 정석(개정판),불안 모범 답안 실습 과제 평가 문항 효능감 복습,10.0,2,취미실용스포츠,취미실용스포츠 불안 모범 답안 실습 과제 평가 문항 효능감 복습


In [50]:
# title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
# sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
title_index = title_book.index.values
title_index

array([3094], dtype=int64)

In [51]:
similar_indexes = genre_sim_sorted_ind[title_index, :10]
similar_indexes2 = genre_sim_sorted_ind2[title_index, :10]

print(similar_indexes)
print(similar_indexes2)

[[3094  192 1059  792 2821  267 2554 1552   36 2678]]
[[3094 1057  671  760  206 1905 1206 2072 2048 2322]]


In [52]:
# 추출된 top_n index들 출력. top_n index는 2차원 데이터 임.
# dataframe에서 index로 사용하기 위해서 2차원 array를 1차원 array로 변경
similar_indexes = similar_indexes.reshape(-1)
similar_indexes2 = similar_indexes2.reshape(-1)
print(similar_indexes)

[3094  192 1059  792 2821  267 2554 1552   36 2678]


###  연금술사와 10개 추천(키워드 기반)

In [54]:
similar_book = find_sim_book_ver1(gyobo_CBF, genre_sim_sorted_ind, '연금술사', 10) # 책 제목 입력
similar_book[['title', 'score', 'total', 'keyword']]
# 문제 ; 평점 기반으로 추천하고자 하는데, vote_count가 낮은 책 제외하고 싶음

[[   1  653 2165 1432 1640  498 2946 1825  309 1234]]


,title,score,total,keyword
1,연금술사,9.6,350,마음 모험 연금술 만물 성장소설 영혼
653,작별하지 않는다,9.3,181,한국소설 삶 마음 사랑 작가 인선
2166,2022 해커스공무원 장재혁 행정법총론 단원별 기출문제집,10.0,14,행정소송 행정 과정 행정 법통 정법 행정쟁송 정상
1432,필라테스 바이블(조셉 필라테스의),10.0,20,마음 로빈스 컨트롤 몸 트위스트 균형
1640,"그냥, 2200km를 걷다",8.8,4,모험 알베르 리스본 순례길 카미노 산티아고
498,정재승의 인간 탐구 보고서. 8: 불안이 온갖 미신을 만든다(어린이를 위한 뇌과학 ...,10.0,44,심리학 뇌과학 마음 외계인 지구인 루이
2947,이렇게 맛있고 멋진 채식이라면. 2: 다이어트가 내 안으로,10.0,2,건강 말린 스무디 슈카 낫토 아보카도
1825,귀멸의 칼날. 22,9.4,177,판타지만화 만화 성장 모험 카마 다이쇼
309,귀멸의 칼날. 12,9.8,217,판타지만화 우정 성장 모험 요리 상현
1234,귀멸의 칼날. 11,9.8,191,판타지만화 우정 성장 모험 도깨비 우즈


###  연금술사와 유사한 책 10개추천(장르추가된 키워드 기반)

In [55]:
similar_book = find_sim_book_ver1(gyobo_CBF, genre_sim_sorted_ind2, '연금술사', 10) # 책 제목 입력
similar_book[['title', 'score', 'total', 'keyword2']]
# 문제 ; 평점 기반으로 추천하고자 하는데, vote_count가 낮은 책은 제외하고 싶음

[[   1 2164 1013  767 3056 2350  653 2659 2020 2885]]


,title,score,total,keyword2
1,연금술사,9.6,350,소설 마음 모험 연금술 만물 성장소설 영혼
2165,라이온의 간식,9.9,32,소설 위로 인생 삶 마음 감동 소설
1013,호수의 일(양장본 HardCover),9.8,32,소설 마음 상처 호정 사춘기 아이 치유 겨울
767,그리스인 조르바(열린책들 세계문학 21)(양장본 HardCover),9.5,150,소설 세계고전문학 우정 성장 장편소설 소설 영혼
3057,쇼코의 미소,9.5,583,소설 소설집 마음 중 단편 소설 젊은작가상 타인 소설가
2351,수레바퀴 아래서(세계문학전집 50),9.5,90,소설 세계고전문학 노벨 문학상 자전 소설 신학교 성장소설 소년
653,작별하지 않는다,9.3,181,소설 한국소설 삶 마음 사랑 작가 인선
2660,자기 앞의 생(문학동네 세계문학)(양장본 HardCover),9.6,269,소설 소설 인생 삶 작가 사랑 모모
2021,어린 왕자(양장본 HardCover),9.7,155,소설 세계고전문학 삶 소설 어린이 작품 번역
2886,불편한 편의점(15만부 기념 윈터 에디션),9.8,634,소설 한국소설 삶 위로 소설 독고 여사


### 가중평점(평점&평가횟수) 반영한추천시스템

In [56]:
# 상위 60%에 해당하는 vote_count를 최소 투표 횟수인 m으로 지정
C = gyobo_CBF['score'].mean()
m = gyobo_CBF['total'].quantile(0.6)

In [57]:
def weighted_vote_average(record):
    v = record['total']
    R = record['score']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )

In [58]:
# 가중평점(weighted_vote) 카테고리 추가
gyobo_CBF['weighted_score'] = gyobo_CBF.apply(weighted_vote_average, axis=1)

In [59]:
# 가중평점(weighted_vote) 기준으로 버터도 달걀도 필요 없는 건강빵과 유사한 책 추천
# 키워드1 기반
similar_movies = find_sim_book_ver2(gyobo_CBF, genre_sim_sorted_ind, '연금술사', 10)
similar_movies[['title', 'score', 'weighted_score', 'keyword', 'total']]

,title,score,weighted_score,keyword,total
498,정재승의 인간 탐구 보고서. 8: 불안이 온갖 미신을 만든다(어린이를 위한 뇌과학 ...,10.0,9.858322,심리학 뇌과학 마음 외계인 지구인 루이,44
1432,필라테스 바이블(조셉 필라테스의),10.0,9.781043,마음 로빈스 컨트롤 몸 트위스트 균형,20
309,귀멸의 칼날. 12,9.8,9.779941,판타지만화 우정 성장 모험 요리 상현,217
1234,귀멸의 칼날. 11,9.8,9.777516,판타지만화 우정 성장 모험 도깨비 우즈,191
2166,2022 해커스공무원 장재혁 행정법총론 단원별 기출문제집,10.0,9.746471,행정소송 행정 과정 행정 법통 정법 행정쟁송 정상,14
2947,이렇게 맛있고 멋진 채식이라면. 2: 다이어트가 내 안으로,10.0,9.629457,건강 말린 스무디 슈카 낫토 아보카도,2
1640,"그냥, 2200km를 걷다",8.8,9.484496,모험 알베르 리스본 순례길 카미노 산티아고,4
1825,귀멸의 칼날. 22,9.4,9.423711,판타지만화 만화 성장 모험 카마 다이쇼,177
653,작별하지 않는다,9.3,9.334956,한국소설 삶 마음 사랑 작가 인선,181


In [60]:
# 가중평점(weighted_vote) 기준으로 버터도 달걀도 필요 없는 건강빵과 유사한 책 추천
# 키워드2 기반
similar_movies = find_sim_book_ver2(gyobo_CBF, genre_sim_sorted_ind2, '연금술사', 10)
similar_movies[['title', 'score', 'weighted_score', 'keyword2', 'total']]

,title,score,weighted_score,keyword2,total
2886,불편한 편의점(15만부 기념 윈터 에디션),9.8,9.792653,소설 한국소설 삶 위로 소설 독고 여사,634
2165,라이온의 간식,9.9,9.770819,소설 위로 인생 삶 마음 감동 소설,32
1013,호수의 일(양장본 HardCover),9.8,9.713676,소설 마음 상처 호정 사춘기 아이 치유 겨울,32
2021,어린 왕자(양장본 HardCover),9.7,9.686402,소설 세계고전문학 삶 소설 어린이 작품 번역,155
2660,자기 앞의 생(문학동네 세계문학)(양장본 HardCover),9.6,9.599884,소설 소설 인생 삶 작가 사랑 모모,269
2351,수레바퀴 아래서(세계문학전집 50),9.5,9.520753,소설 세계고전문학 노벨 문학상 자전 소설 신학교 성장소설 소년,90
767,그리스인 조르바(열린책들 세계문학 21)(양장본 HardCover),9.5,9.513597,소설 세계고전문학 우정 성장 장편소설 소설 영혼,150
3057,쇼코의 미소,9.5,9.503898,소설 소설집 마음 중 단편 소설 젊은작가상 타인 소설가,583
653,작별하지 않는다,9.3,9.334956,소설 한국소설 삶 마음 사랑 작가 인선,181
